# NYC Traffic Project

## Data Cleaning

In [66]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [1]:
# NYCTrafficDanger
#NYC Traffic Danger Project for NYU Programming for Data Science Class

#Load data
#Starting number of rows: 1,945,288
df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes_Raw_Data.csv')

#Convert CRASH DATE to pandas datetime object 
df['CRASH DATE']= pd.to_datetime(df['CRASH DATE'])

#Filter dataframe to 2021
filtered_df = df.loc[(df['CRASH DATE'] >= '2021-01-01') & (df['CRASH DATE'] <= '2021-12-31')]
#Number of rows after filtering by 2021: 110,541

#Drop all rows with null LOCATION data 
df2 = filtered_df[filtered_df['LOCATION'].notna()]
#Number of rows after filtering by LOCATION is not empty: 101,757

#All data (cars and all bikes)
#Number of rows: 101,757
df3 = df2.drop(['ON STREET NAME','CROSS STREET NAME', 'OFF STREET NAME','NUMBER OF PEDESTRIANS INJURED','NUMBER OF PEDESTRIANS KILLED','NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED'], axis=1)
df5 = df2.drop(['ON STREET NAME','CROSS STREET NAME', 'OFF STREET NAME'], axis=1)

#Find unique labels for vehicle type for each vehicle involved in crash
columns = ['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4','VEHICLE TYPE CODE 5']
for col in columns:
    df4 = df3[col].unique()
    #print(df4)

#Bike data (bikes and e-bikes)
#Number of rows: 7,331
bike = df3.loc[(df3['VEHICLE TYPE CODE 1']=='Bike') | (df3['VEHICLE TYPE CODE 1']=='E-Bike')| (df3['VEHICLE TYPE CODE 1']=='E BIKE') |(df3['VEHICLE TYPE CODE 1']=='E-BIKE')| (df3['VEHICLE TYPE CODE 1']=='E-bike')| (df3['VEHICLE TYPE CODE 2']=='Bike') | (df3['VEHICLE TYPE CODE 2']=='E-Bike')| (df3['VEHICLE TYPE CODE 2']=='E BIKE') |(df3['VEHICLE TYPE CODE 2']=='E-BIKE')| (df3['VEHICLE TYPE CODE 2']=='E-bike')|(df3['VEHICLE TYPE CODE 3']=='Bike') | (df3['VEHICLE TYPE CODE 3']=='E-Bike')| (df3['VEHICLE TYPE CODE 3']=='E BIKE') |(df3['VEHICLE TYPE CODE 3']=='E-BIKE')| (df3['VEHICLE TYPE CODE 3']=='E-bike')|(df3['VEHICLE TYPE CODE 4']=='Bike') | (df3['VEHICLE TYPE CODE 4']=='E-Bike')| (df3['VEHICLE TYPE CODE 4']=='E BIKE') |(df3['VEHICLE TYPE CODE 4']=='E-BIKE')| (df3['VEHICLE TYPE CODE 4']=='E-bike')|(df3['VEHICLE TYPE CODE 5']=='Bike') | (df3['VEHICLE TYPE CODE 5']=='E-Bike')| (df3['VEHICLE TYPE CODE 5']=='E BIKE') |(df3['VEHICLE TYPE CODE 5']=='E-BIKE')| (df3['VEHICLE TYPE CODE 5']=='E-bike')]

df3.to_csv('All clean data.csv')
bike.to_csv('clean bike data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Motor_Vehicle_Collisions_-_Crashes_Raw_Data.csv'

### Filling in General Location

In [ ]:
clean_all2 = df5
clean_all2['LOCATION'] = clean_all2['LOCATION'].str.replace(r"[()]",'') 
clean_all2

In [ ]:
geolocator = Nominatim(user_agent='school project')

startTime = time.perf_counter()

#Write in code to skip the lines that don't have a postcode

boolean_null = clean_all2['BOROUGH'].isna()


#len(clean_all2['LOCATION']) clean_all2['ZIP CODE'].iloc[i] != 11233.0
for i in range(len(clean_all2['LOCATION'])):
    if boolean_null.iloc[i] == True:
        reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
        location = reverse(clean_all2['LOCATION'].iloc[i]) #changed from location = geolocator.reverse(clean_bike['LOCATION'].iloc[i])
        print(i)
        try:
            postcode = location.raw['address']['postcode']
            clean_all2['ZIP CODE'].iloc[i] = postcode
        except KeyError:
            clean_all2['ZIP CODE'].iloc[i] = "error"
        try:
            suburbRaw = location.raw['address']['suburb']
            clean_all2['BOROUGH'].iloc[i] = suburbRaw
        except KeyError:
            clean_all2['BOROUGH'].iloc[i] = "error"

endTime = time.perf_counter()

clean_all2

In [ ]:
#clean_all_with_zips_buroughs = pd.read_csv('clean_all_with_zips_buroughs.csv')
clean_all2 = clean_all2[clean_all2['ZIP CODE'] != 'error']
clean_all2 = clean_all2[clean_all2['BOROUGH'] != 'error']

In [ ]:
#Make all values upper case in new borough column
clean_all2['BOROUGH'] = clean_all2['BOROUGH'].str.upper()

#Standardize values for the new borough column
for i in range(len(clean_all2['BOROUGH'])):
    if clean_all2['BOROUGH'].iloc[i] == 'QUEENS COUNTY': 
        clean_all2['BOROUGH'].iloc[i] = 'QUEENS'
    elif clean_all2['BOROUGH'].iloc[i] == 'THE BRONX':
        clean_all2['BOROUGH'].iloc[i] = 'BRONX'
    elif clean_all2['BOROUGH'].iloc[i] == 'KINGS COUNTY':
        clean_all2['BOROUGH'].iloc[i] = 'BROOKLYN'
    elif clean_all2['BOROUGH'].iloc[i] == 'RICHMOND COUNTY':
        clean_all2['BOROUGH'].iloc[i] = 'STATEN ISLAND'

clean_all2.to_csv('clean_all_with_zips_buroughs_final_version.csv')

### Filling in Bike Location Data

In [46]:
#Read in data from clean data files csv as dataframes
clean_all = pd.read_csv('All clean data.csv')
clean_bike = pd.read_csv('clean bike data.csv')

#Remove parentheses from location data column
for i in range(len(clean_bike['LOCATION'])):
    clean_bike['LOCATION'].loc[i] = (clean_bike['LOCATION'].loc[i][1:-1])
    
clean_bike.head()

/var/folders/jt/l0pnll0d6rs6dfjgfd6y8f0w0000gn/T/ipykernel_58481/3605484986.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_bike['LOCATION'].loc[i] = (clean_bike['LOCATION'].loc[i][1:-1])


,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"40.687534, -73.9775",1.0,0.0,...,Unspecified,NaN,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN
1,31,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"40.784615, -73.953964",1.0,0.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4486581,Van,Bike,NaN,NaN,NaN
2,65,2021-06-29,17:35,NaN,NaN,40.609535,-73.753720,"40.609535, -73.75372",0.0,0.0,...,Unspecified,NaN,NaN,NaN,4456672,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN
3,66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"40.59207, -73.96299",1.0,0.0,...,Unspecified,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN
4,72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"40.640835, -73.98967",1.0,0.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN


In [66]:
#Find any values that do not have valid location data and their indices
for i in range(len(clean_bike['LOCATION'])):
    if len(clean_bike['LOCATION'][i]) < 15: 
        print(i, clean_bike['LOCATION'][i] )

232 0.0, 0.0
394 0.0, 0.0
894 0.0, 0.0
932 0.0, 0.0
1237 0.0, 0.0
1560 0.0, 0.0
1648 0.0, 0.0
1809 0.0, 0.0
1843 0.0, 0.0
1854 0.0, 0.0
1861 0.0, 0.0
1871 0.0, 0.0
1877 0.0, 0.0
2013 0.0, 0.0
2454 0.0, 0.0
2880 0.0, 0.0
2883 0.0, 0.0
2886 0.0, 0.0
2891 0.0, 0.0
2893 0.0, 0.0
2897 0.0, 0.0
2913 0.0, 0.0
2914 0.0, 0.0
3029 0.0, 0.0
3404 0.0, 0.0
3487 0.0, 0.0
4546 0.0, 0.0
5067 0.0, 0.0
5073 0.0, 0.0
5109 0.0, 0.0
5415 0.0, 0.0
5697 0.0, 0.0
5813 0.0, 0.0
5873 0.0, 0.0
5949 0.0, 0.0
6117 0.0, 0.0
6213 0.0, 0.0
6317 0.0, 0.0
6318 0.0, 0.0
6326 0.0, 0.0
6351 0.0, 0.0
6384 0.0, 0.0
6444 0.0, 0.0
6445 0.0, 0.0
6482 0.0, 0.0
6509 0.0, 0.0
6511 0.0, 0.0
6527 0.0, 0.0
6547 0.0, 0.0
6567 0.0, 0.0
6576 0.0, 0.0
6588 0.0, 0.0
6590 0.0, 0.0
6600 0.0, 0.0
6614 0.0, 0.0
6619 0.0, 0.0
6625 0.0, 0.0
6659 0.0, 0.0
6670 0.0, 0.0
6683 0.0, 0.0
6703 0.0, 0.0
6714 0.0, 0.0
6741 0.0, 0.0
6752 0.0, 0.0
6755 0.0, 0.0
6756 0.0, 0.0
6762 0.0, 0.0
6786 0.0, 0.0
6828 0.0, 0.0
6831 0.0, 0.0
6872 0.0, 0.0
6889 0.0, 

In [69]:
#Remove 0s from the location column
#There were 77 rows with 0s in the location column
clean_bike = clean_bike[(clean_bike['LOCATION']!='0.0, 0.0')]

In [65]:
#Example case for geolocation and check for error location
location = geolocator.reverse(clean_bike['LOCATION'].iloc[1912])
location.raw
#location.raw['address']['postcode']

NameError: name 'geolocator' is not defined

In [113]:
#This block takes ~an hour to run due to the nominatim API that it is pulling from
#Pull data from nominatim for all rows in the database
geolocator = Nominatim(user_agent='school project')

#Track start time of program
startTime = time.perf_counter()

#Create empty arrays to be filled with zipcodes and burough for all collision points
zipcodes = []
suburb = []

#Function to skip the lines that don't have a postcode and put an error in their place
#1911 is the first instance that does not have a postcode
#location.raw reads all information that comes out of the location data
for i in range(len(clean_bike['LOCATION'])):
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    location = reverse(clean_bike['LOCATION'].iloc[i]) 
    print(i)
    try:
        postcode = location.raw['address']['postcode']
        zipcodes.append(postcode)
    except KeyError:
        zipcodes.append('error')
    try:
        suburbRaw = location.raw['address']['suburb']
        suburb.append(suburbRaw)
    except KeyError:
        suburb.append('error')

endTime = time.perf_counter()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977


In [116]:
#Time to run the code for finding zip code and borough information
startTime-endTime

-3649.922324957999

In [114]:
#Add zipcodes and boroughs to their own columns in the database
clean_bike['Zipcode Added'] = zipcodes
clean_bike['Borough Added'] = suburb

clean_bike.head()

,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,Zipcode Added,Borough Added
0,27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"40.687534, -73.9775",1.0,0.0,...,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN,11207,Brooklyn
1,31,2021-12-14,16:25,NaN,NaN,40.784615,-73.953964,"40.784615, -73.953964",1.0,0.0,...,NaN,NaN,4486581,Van,Bike,NaN,NaN,NaN,10128,Manhattan
2,65,2021-06-29,17:35,NaN,NaN,40.609535,-73.753720,"40.609535, -73.75372",0.0,0.0,...,NaN,NaN,4456672,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,11691,Queens County
3,66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"40.59207, -73.96299",1.0,0.0,...,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN,11223,Brooklyn
4,72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"40.640835, -73.98967",1.0,0.0,...,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN,11219,Brooklyn


In [43]:
#Save new cleaned data to csv file to be used for future pulls so that the lengthy code doesn't have to run each time
clean_bike.to_csv('clean_bike_with_zips_buroughs.csv')

NameError: name 'clean_bike' is not defined

In [54]:
#Eliminate errors within the zipcode file: 
#6 rows did not have zipcode data in the Nominatim returned dictionary and are eliminated here
clean_bike_with_zips_buroughs = pd.read_csv('clean_bike_with_zips_buroughs.csv')
clean_bike_with_zips_buroughs = clean_bike_with_zips_buroughs[clean_bike_with_zips_buroughs['Zipcode Added'] != 'error']
clean_bike_with_zips_buroughs = clean_bike_with_zips_buroughs[clean_bike_with_zips_buroughs['Borough Added'] != 'error']

In [55]:
#Make all values upper case in new borough column
clean_bike_with_zips_buroughs['Borough Added'] = clean_bike_with_zips_buroughs['Borough Added'].str.upper()

#Read out all unique value of the new column and borough column
print(clean_bike_with_zips_buroughs['Borough Added'].unique())
print(clean_bike_with_zips_buroughs['BOROUGH'].unique())

#Standardize values for the new borough column, ex: change 'queens county' to 'queens'
for i in range(len(clean_bike_with_zips_buroughs['Borough Added'])):
    if clean_bike_with_zips_buroughs['Borough Added'].iloc[i] == 'QUEENS COUNTY': 
        clean_bike_with_zips_buroughs['Borough Added'].iloc[i] = 'QUEENS'
    elif clean_bike_with_zips_buroughs['Borough Added'].iloc[i] == 'THE BRONX':
        clean_bike_with_zips_buroughs['Borough Added'].iloc[i] = 'BRONX'
    elif clean_bike_with_zips_buroughs['Borough Added'].iloc[i] == 'KINGS COUNTY':
        clean_bike_with_zips_buroughs['Borough Added'].iloc[i] = 'BROOKLYN'
    elif clean_bike_with_zips_buroughs['Borough Added'].iloc[i] == 'RICHMOND COUNTY':
        clean_bike_with_zips_buroughs['Borough Added'].iloc[i] = 'STATEN ISLAND'

#Create column of true/false to check if our values for borough match nyc database values
clean_bike_with_zips_buroughs['Borough Matching Col'] = clean_bike_with_zips_buroughs['BOROUGH'] == clean_bike_with_zips_buroughs['Borough Added']

#drop the rows containing null values in the nyc database borough columns 
borough_check = clean_bike_with_zips_buroughs.dropna(subset = ['BOROUGH'])

#Find number of cells that do not match the original borough input from nyc dataset
#7 rows have different boroughs between geopy and nyc database
borough_check[borough_check['Borough Matching Col']==False]

['BROOKLYN' 'MANHATTAN' 'QUEENS COUNTY' 'STATEN ISLAND' 'QUEENS'
 'THE BRONX' 'KINGS COUNTY' 'RICHMOND COUNTY']
['BROOKLYN' nan 'MANHATTAN' 'STATEN ISLAND' 'QUEENS' 'BRONX']


/var/folders/jt/l0pnll0d6rs6dfjgfd6y8f0w0000gn/T/ipykernel_63710/3360292903.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_bike_with_zips_buroughs['Borough Added'].iloc[i] = 'QUEENS'
/var/folders/jt/l0pnll0d6rs6dfjgfd6y8f0w0000gn/T/ipykernel_63710/3360292903.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_bike_with_zips_buroughs['Borough Added'].iloc[i] = 'BRONX'
/var/folders/jt/l0pnll0d6rs6dfjgfd6y8f0w0000gn/T/ipykernel_63710/3360292903.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

,Unnamed: 0.1,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,Zipcode Added,Borough Added,Borough Matching Col
841,843,13672,2021-12-14,13:30,QUEENS,11385.0,40.707714,-73.919044,"40.707714, -73.919044",0.0,...,NaN,4488894,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,11237,BROOKLYN,False
1324,1329,21686,2021-05-20,14:53,BROOKLYN,11237.0,40.699436,-73.912290,"40.699436, -73.91229",1.0,...,NaN,4418853,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,11385,QUEENS,False
1743,1750,28352,2021-05-30,0:10,BROOKLYN,11237.0,40.696117,-73.905014,"40.696117, -73.905014",1.0,...,NaN,4422301,Bike,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,11385,QUEENS,False
1931,1944,31619,2021-10-02,22:09,BRONX,10463.0,40.873910,-73.909164,"40.87391, -73.909164",1.0,...,NaN,4464834,Sedan,E-Bike,NaN,NaN,NaN,10463,MANHATTAN,False
2324,2338,37669,2021-06-13,21:30,QUEENS,11101.0,40.742230,-73.958850,"40.74223, -73.95885",1.0,...,NaN,4427088,E-Bike,NaN,NaN,NaN,NaN,11109,MANHATTAN,False
4922,4949,80077,2021-10-14,21:05,MANHATTAN,10002.0,40.714945,-73.976650,"40.7149447, -73.9766502",1.0,...,NaN,4467198,E-Bike,NaN,NaN,NaN,NaN,11207,BROOKLYN,False
4981,5008,81038,2021-08-22,20:51,BRONX,10463.0,40.875683,-73.908390,"40.875683, -73.90839",1.0,...,NaN,4450213,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN,10463,MANHATTAN,False


In [56]:
#Testing the given zip codes against our calculated zip codes, returning a percentage of those that are different:
clean_bike_zip_compare = clean_bike[clean_bike['Zipcode Added'] != 'error']
clean_bike_zip_compare = clean_bike_zip_compare.dropna(subset=['ZIP CODE'])
clean_bike_zip_compare['Zipcode Added'] = pd.to_numeric(clean_bike_zip_compare['Zipcode Added'])
clean_bike_zip_compare['Zipcode Compare'] = clean_bike_zip_compare['Zipcode Added'] == clean_bike_zip_compare['ZIP CODE']
(clean_bike_zip_compare['Zipcode Compare'].count() - clean_bike_zip_compare['Zipcode Compare'].sum())/clean_bike_zip_compare['Zipcode Compare'].count()

NameError: name 'clean_bike' is not defined

In [57]:
#Sub in values from Borough Added column to original Borough column for NaN Values
clean_bike_with_zips_buroughs['BOROUGH'].fillna(clean_bike_with_zips_buroughs['Borough Added'], inplace=True)
del clean_bike_with_zips_buroughs['Borough Added']
del clean_bike_with_zips_buroughs['Borough Matching Col']
clean_bike_with_zips_buroughs

#Sub in values from Zipcode Added column to original Zip Code column for NaN Values

,Unnamed: 0.1,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,Zipcode Added
0,0,27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"40.687534, -73.9775",1.0,...,NaN,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN,11207
1,1,31,2021-12-14,16:25,MANHATTAN,NaN,40.784615,-73.953964,"40.784615, -73.953964",1.0,...,NaN,NaN,NaN,4486581,Van,Bike,NaN,NaN,NaN,10128
2,2,65,2021-06-29,17:35,QUEENS,NaN,40.609535,-73.753720,"40.609535, -73.75372",0.0,...,NaN,NaN,NaN,4456672,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,11691
3,3,66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"40.59207, -73.96299",1.0,...,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN,11223
4,4,72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"40.640835, -73.98967",1.0,...,NaN,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN,11219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,7326,186069,2021-01-27,7:00,MANHATTAN,NaN,40.786808,-73.945240,"40.786808, -73.94524",1.0,...,NaN,NaN,NaN,4387268,Tanker,E-Bike,NaN,NaN,NaN,10029
7250,7327,186131,2021-01-22,22:45,BROOKLYN,11218.0,40.643574,-73.971660,"40.643574, -73.97166",2.0,...,Unspecified,NaN,NaN,4386418,Sedan,Sedan,Bike,NaN,NaN,11218
7251,7328,186178,2021-02-06,0:30,MANHATTAN,10002.0,40.720980,-73.986960,"40.72098, -73.98696",1.0,...,NaN,NaN,NaN,4391810,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,10002
7252,7329,186185,2021-01-06,11:02,BROOKLYN,11226.0,40.650753,-73.950750,"40.650753, -73.95075",0.0,...,NaN,NaN,NaN,4382028,LIMO,Bike,NaN,NaN,NaN,11226


In [59]:
#Sub in values from Zipcode Added column to original Zip Code column for NaN Values
#Change values to floats before moving them to ZIP CODE column
clean_bike_with_zips_buroughs['ZIP CODE'].fillna(pd.to_numeric(clean_bike_with_zips_buroughs['Zipcode Added']), inplace=True)
del clean_bike_with_zips_buroughs['Zipcode Added']
clean_bike_with_zips_buroughs

,Unnamed: 0.1,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,0,27,2021-12-14,12:54,BROOKLYN,11217.0,40.687534,-73.977500,"40.687534, -73.9775",1.0,...,Unspecified,NaN,NaN,NaN,4487052,Sedan,Bike,NaN,NaN,NaN
1,1,31,2021-12-14,16:25,MANHATTAN,10128.0,40.784615,-73.953964,"40.784615, -73.953964",1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4486581,Van,Bike,NaN,NaN,NaN
2,2,65,2021-06-29,17:35,QUEENS,11691.0,40.609535,-73.753720,"40.609535, -73.75372",0.0,...,Unspecified,NaN,NaN,NaN,4456672,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN
3,3,66,2021-12-09,20:20,BROOKLYN,11223.0,40.592070,-73.962990,"40.59207, -73.96299",1.0,...,Unspecified,NaN,NaN,NaN,4485150,Bike,NaN,NaN,NaN,NaN
4,4,72,2021-12-09,23:15,BROOKLYN,11218.0,40.640835,-73.989670,"40.640835, -73.98967",1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4485355,Sedan,Bike,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7249,7326,186069,2021-01-27,7:00,MANHATTAN,10029.0,40.786808,-73.945240,"40.786808, -73.94524",1.0,...,Unspecified,NaN,NaN,NaN,4387268,Tanker,E-Bike,NaN,NaN,NaN
7250,7327,186131,2021-01-22,22:45,BROOKLYN,11218.0,40.643574,-73.971660,"40.643574, -73.97166",2.0,...,Unspecified,Unspecified,NaN,NaN,4386418,Sedan,Sedan,Bike,NaN,NaN
7251,7328,186178,2021-02-06,0:30,MANHATTAN,10002.0,40.720980,-73.986960,"40.72098, -73.98696",1.0,...,Unspecified,NaN,NaN,NaN,4391810,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN
7252,7329,186185,2021-01-06,11:02,BROOKLYN,11226.0,40.650753,-73.950750,"40.650753, -73.95075",0.0,...,Unspecified,NaN,NaN,NaN,4382028,LIMO,Bike,NaN,NaN,NaN


In [60]:
#Save final cleaned bike data file to csv
clean_bike_with_zips_buroughs.to_csv('clean_bike_final.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f5d584d7-8c21-4a43-bee6-b148c3c0a755' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>